In [2]:
from rdflib import Graph
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

output = widgets.Output()
instance_file_path = None
result_graph = None

# Auswahlfeld
option_selector = widgets.ToggleButtons(
    options=[("Use example", "example"), ("Upload your own", "upload")],
    description="Select source:",
    style={'description_width': 'initial'}
)

# Upload-Widget
upload_widget = widgets.FileUpload(accept='.ttl', multiple=False)

# Continue Button
continue_button = widgets.Button(description="Continue", button_style='info')

# Auswahl-Handler
def on_option_change(change):
    output.clear_output()
    with output:
        if change['new'] == 'example':
            global instance_file_path
            instance_file_path = "Instance_Files/swemls-instances.ttl"
            print(f"✅ Example file selected: {instance_file_path}")
        elif change['new'] == 'upload':
            print("📁 Please upload your TTL file below:")
    display(upload_widget)

option_selector.observe(on_option_change, names='value')

# Upload-Handler
def on_upload_change(change):
    global instance_file_path
    upload_output = list(upload_widget.value.values())[0]
    instance_file_path = upload_output['metadata']['name']
    with open(instance_file_path, 'wb') as f:
        f.write(upload_output['content'])
    output.clear_output()
    with output:
        print(f"✅ File uploaded and saved as: {instance_file_path}")

upload_widget.observe(on_upload_change, names='value')

# Weiter-Button-Handler
def on_continue_clicked(b):
    global result_graph
    output.clear_output()

    if not instance_file_path or not os.path.exists(instance_file_path):
        with output:
            print("❌ No file selected or file not found.")
        return

    result_graph = Graph()
    try:
        result_graph.parse(instance_file_path, format="turtle")
    except Exception as e:
        with output:
            print(f"❌ Failed to parse TTL file: {e}")
        return

    with output:
        print(f"✅ Parsed file: {instance_file_path}")
        print(f"📊 Triple count: {len(result_graph)}")

continue_button.on_click(on_continue_clicked)

# UI anzeigen
display(option_selector, output, continue_button)


ModuleNotFoundError: No module named 'rdflib'